* Read Data
* Make Dataset of `imagepath` and `labels`
* Convert images into tensors
* Test-Train-Split
* Train a DenseNet121 model
* Test the model
* Get Metrics
* ROC Curve

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.layers import Dense, Flatten, Input, Lambda
from tensorflow import keras

## **Read Data**

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MIAS-Classification/Info.txt",sep=" ")
data

In [ ]:
data.SEVERITY.fillna("H", inplace=True)
data.SEVERITY = data.SEVERITY.map({'H':0, 'B':1, 'M':2})

In [ ]:
data.SEVERITY.unique()

In [ ]:
_ = [ 'BG', 'CLASS', 'X', 'Y', 'RADIUS', 'Unnamed: 7']
data = data.drop(_, axis = 1)
data.info()

In [ ]:
data = data.drop([5], axis=0)
data.reset_index(inplace = True)
data = data.drop("index",axis=1)
data

## **Make Dataset of ```imagepath``` and ```labels```**






In [ ]:
ohe = OneHotEncoder()
img_labels = ohe.fit_transform(data.SEVERITY.values.reshape(-1, 1)).toarray()

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/MIAS-Classification/all-mias/"
img_paths = []
# img_labels = []

for i in range(len(data)):
  abs_path = path + data.REFNUM[i] + '.pgm'
  img_paths.append(abs_path)
  # img_labels.append(data.SEVERITY[i])

In [ ]:
img_paths[0], img_labels[0]

In [ ]:
image_numeric = []
for img in img_paths:
  image = cv2.imread(img)
  image = cv2.resize(image, (224,224)) # put `/255` to normalize the colour scores
  image_numeric.append(image)

In [ ]:
image_numeric = np.array(image_numeric)/255.0
img_labels = np.array(img_labels)

In [ ]:
image_numeric[0].shape

In [ ]:
img_labels.shape

## **Train-Test-Split**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(image_numeric, img_labels, test_size = 0.3, random_state = 42)

In [ ]:
# y_train=y_train.reshape(-1,1)

In [ ]:
print(x_train.shape) #, y_train.shape, x_test.shape, y_test.shape
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# x_train = x_train[..., np.newaxis]

In [ ]:
x_train.shape

## **Train a DenseNet121 Model**

In [ ]:
%%time
base_model = DenseNet121(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False
)

base_model.trainable = False

# Create inputs with correct shape
inputs = Input(shape = (224,224,3))

x = base_model(inputs, training=False)
# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)


# Add final dense layer
outputs = Dense(3, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

model.compile(loss = keras.losses.CategoricalCrossentropy() , metrics = ['accuracy']) # compiles the model with the given parameters

model.summary() # prints a summary of the model

In [ ]:
%%time
EPOCH = 25
d_model = model.fit(x_train, y_train, epochs = EPOCH, validation_data = (x_test, y_test) )

## **Confusion Matrix and Classification Report**

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# loss
fig, ax = plt.subplots()
ax.plot(d_model.history['loss'], label='loss')
ax.plot(d_model.history['val_loss'], label='val_loss')
ax.legend()
plt.title('DenseNet121')
plt.show()
fig.savefig('/content/drive/MyDrive/Colab Notebooks/MIAS-Classification/DenseNet121Loss.png')

In [ ]:
# accuracies
fig, ax = plt.subplots()
ax.plot(d_model.history['accuracy'], label='acc')
ax.plot(d_model.history['val_accuracy'], label='val acc')
ax.legend()
plt.title('DenseNet121')
plt.show()
fig.savefig('/content/drive/MyDrive/Colab Notebooks/MIAS-Classification/DenseNet121Accuracy.png')

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/MIAS-Classification/Densenet121.h5")

In [ ]:
y_preds =  model.predict(x_test)
y_preds

In [ ]:
y_preds_ind = np.argmax(y_preds,axis=1)
y_preds_ind

In [ ]:
qwe=np.argmax(y_test, axis=1)

In [ ]:
confusion_matrix(qwe,y_preds_ind)

In [ ]:
print(classification_report(qwe,y_preds_ind))

In [ ]:
model.evaluate(x_test,y_test)

## **ROC Curve**

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

fpr, tpr, thresholds = roc_curve(qwe, y_preds_ind, pos_label=2)

auc = auc(fpr, tpr)

import matplotlib.pyplot as plt
plt.figure(1)
plt.plot([0, 1], [0, 1],"--")

plt.plot(fpr, tpr, label='RF (area = {:.3f})'.format(auc), color='orange')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

### **Visualization**

In [ ]:
def Visualize_Result(acc,val_acc,loss, val_loss):
    fig, (ax1, ax2) = plt.subplots(nrows = 1,
                                   ncols = 2,
                                   figsize = (15,6))

    plot1 = ax1.plot(range(0, len(acc)),
                     acc,
                     label = 'accuracy')

    plot2 = ax1.plot(range(0, len(val_acc)),
                     val_acc,
                     label = 'val_accuracy')

    ax1.set(title = 'Accuracy And Val Accuracy progress',
            xlabel = 'epoch',
            ylabel = 'accuracy/ validation accuracy')

    ax1.legend()

    plot3 = ax2.plot(range(0, len(loss)),
                     loss,
                     label = 'loss')
    
    plot4 = ax2.plot(range(0, len(val_loss)),
                     val_loss,
                     label = 'val_loss')
    
    ax2.set(title = 'Loss And Val loss progress',
            xlabel = 'epoch',
            ylabel = 'loss/ validation loss')

    ax2.legend()

    fig.suptitle('Result Of Model', fontsize = 20, fontweight = 'bold')
    fig.savefig('/content/drive/MyDrive/Colab Notebooks/MIAS-Classification/Accuracy_Loss_figure.png')
    plt.tight_layout()
    plt.show()

visualize_result = Visualize_Result(d_model.history['accuracy'],d_model.history['val_accuracy'], d_model.history['loss'], d_model.history['val_loss'])


## **Cross-Validation**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
#set early stopping criteria
pat = 5 #this is the number of epochs with no improvment after which the training will stop
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

#define the model checkpoint callback -> this will keep on saving the model as a physical file
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/MIAS-Classification/Models/h_mias.h5', verbose=1, save_best_only=True)

#define a function to fit the model
def fit_and_evaluate(x_train, x_test, y_train, y_test, EPOCHS=20, BATCH_SIZE=32,model=model):
    results = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping, model_checkpoint], 
                        verbose=1, validation_split=0.3)  
    print("Val Score: ", model.evaluate(x_test,y_test))
    return results

In [ ]:
n_folds=10
epochs=20
batch_size=32

#save the model history after fitting so that we can plot later
model_history = [] 

for i in range(n_folds):
    print("Training on Fold: ",i+1)
    x_train, x_test, y_train, y_test = train_test_split(image_numeric, img_labels, test_size = 0.3, 
                                               random_state = np.random.randint(1,1000, 1)[0])
    model_history.append(fit_and_evaluate(x_train, x_test, y_train, y_test, epochs, batch_size))
    print("======="*12, end="\n\n\n")

In [ ]:
plt.title('Accuracies vs Epochs')
for i in range(len(model_history)):
  plt.plot(model_history[i].history['accuracy'], label=f"Training Fold {i+1}")

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Train Accuracy vs Val Accuracy')
plt.plot(model_history[0].history['accuracy'], label='Train Accuracy Fold 1', color='black')
plt.plot(model_history[0].history['val_accuracy'], label='Val Accuracy Fold 1', color='black', linestyle = "dashdot")
plt.plot(model_history[1].history['accuracy'], label='Train Accuracy Fold 2', color='red', )
plt.plot(model_history[1].history['val_accuracy'], label='Val Accuracy Fold 2', color='red', linestyle = "dashdot")
plt.plot(model_history[2].history['accuracy'], label='Train Accuracy Fold 3', color='green', )
plt.plot(model_history[2].history['val_accuracy'], label='Val Accuracy Fold 3', color='green', linestyle = "dashdot")
plt.legend()
plt.show()

In [ ]:
# plt.figure(figsize=(15,10))
# plt.title('Train Accuracy vs Val Accuracy')
# for i in range(len(model_history)):
#   plt.plot(model_history[i].history['accuracy'], label=f"Trai Accuracy Fold {i+1}")
#   plt.plot(model_history[i].history['val_accuracy'], label=f'Val Accuracy Fold {i+1}', linestyle = "dashdot")

# plt.legend()
# plt.show()

In [ ]:
#Load the model that was saved by ModelCheckpoint
from tensorflow import keras

model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/MIAS-Classification/Models/h_mias.h5')

model.evaluate(x_test,y_test)

In [ ]:
y_preds =  model.predict(x_test)
y_preds

In [ ]:
y_preds_arg = np.argmax(y_preds,axis=1)
y_preds_arg

In [ ]:
asd = np.argmax(y_test, axis=1)
asd

In [ ]:
confusion_matrix(asd,y_preds_arg)

In [ ]:
print(classification_report(asd,y_preds_arg))

In [ ]:
#function to draw confusion matrix
import seaborn as sns

def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,fmt='g', cmap="viridis")
    plt.show()

In [ ]:
draw_confusion_matrix(asd,y_preds_arg)